Let's load up our data and do a little cleanup. We have three columns that are always null, so let's drop them. We also have one column that always contains the value True, so let's drop it too.

In [1]:
import pandas as pd

DATA = '/kaggle/input/jollibee-fastfood-store-locations/jollibee-branches.csv'
df = pd.read_csv(filepath_or_buffer=DATA).drop(columns=['url', 'media_url', 'maphub_image_url', 'visible'])
df.head()

,title,description,group,icon,color,longitude,latitude
0,5th Avenue,NaN,Caloocan,NaN,#005e38,120.983717,14.643881
1,9th Avenue,NaN,Caloocan,NaN,#005e38,120.984059,14.650442
2,Grace Park,NaN,Caloocan,NaN,#005e38,120.984088,14.654215
3,Araneta Square,NaN,Caloocan,NaN,#005e38,120.983555,14.656720
4,Victory Central Mall,NaN,Caloocan,NaN,#005e38,120.982529,14.655712


Let's make a map. Choosing the name, data, and location is easy, but it's not clear what we should do for color. First let's try using the color field.

In [2]:
from plotly import express
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)

express.scatter_mapbox(mapbox_style='open-street-map', height=800, zoom=4.25, data_frame=df, lat='latitude', lon='longitude', hover_name='title', hover_data=['group'], color='color').show(renderer='iframe_connected',)

This makes a nice map, but the legend doesn't really tell us anything.

In [3]:
df['color'].value_counts().to_frame().T

color,#005e38,#7478fb,#660000,#14f6f1,#020256,#ff7373,#482d9e,#e32c31,#894411,#9e9e9e,#f18729,#2d2d2d,#64e968,#ffcc00,#b63681,#3cc954,#cfd965,#00c4bf,#000000,#00ff98
count,379,205,140,90,60,48,46,42,32,30,28,28,24,21,20,16,15,12,6,3


The colors are distributed unevenly, but they don't have any intuitive meaning. How about the group?

In [4]:
df['group'].value_counts().to_frame().T

group,Quezon City,Manila,Cavite,Laguna,Cebu City - Lapu-Lapu - Mandaue,Bulacan,Batangas,Central Visayas,Rizal,Bicol,...,Malabon,Ilocos Sur,NAIA,Ilocos Norte,Nueva Vizcaya,Bangsamoro,Navotas,Pateros,Aurora,Malaysia
count,95,66,58,54,54,39,39,36,36,32,...,8,7,5,5,3,3,2,1,1,1


In [5]:
df['group'].nunique()

61

We have a lot more groups, and they make some sort of geographic sense, but we have a lot of them. Let's try coloring by group and see what happens.

In [6]:
n_colors = df['group'].nunique()
colors = express.colors.sample_colorscale(colorscale='turbo', samplepoints=[n/(n_colors -1) for n in range(n_colors)])


express.scatter_mapbox(color_discrete_sequence=colors, mapbox_style='open-street-map', height=800, zoom=4.25, data_frame=df, lat='latitude', lon='longitude', hover_name='title', hover_data=['group'], color='group').show(renderer='iframe_connected',)

That's not bad. We have a color scale with a lot of colors, and some of them are hard to tell apart, but it's hard to do otherwise with that many colors.